In [19]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm

In [20]:
from pathlib import Path

# Get the current working directory
current_path = Path.cwd()

# Get the parent directory
parent_path = current_path
data_paths = parent_path / 'datas'  

print(data_paths)
#
train_path = data_paths / 'train.csv'
test_path = data_paths / 'test.csv'

train_path_ts = data_paths / 'series_train.parquet'
test_path_ts = data_paths / 'series_test.parquet'

/home/koko/Bureau/projets/kaggle/cmi_probleamtic_use/CMI_problematic_internet_use/datas


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import optuna
from optuna import Trial, create_study
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Load datasets

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

## Pre processing

## Strategy

We can make category per Age/sex

for float take median 

In [23]:
features_to_keep_test = ['id', 'Basic_Demos-Age', 'Basic_Demos-Sex','CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Diastolic_BP','Physical-HeartRate','Physical-Systolic_BP','Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Sec', 
                    'FGC-FGC_CU_Zone','FGC-FGC_GSND_Zone','FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone','FGC-FGC_TL_Zone',
                    'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                    'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num','BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW',
                    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday']

features_to_keep_train = np.concatenate((features_to_keep_test,['sii']))
                         
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import optuna
from optuna import Trial, create_study
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [24]:
def detect_outliers(df, n_std=3):
    """
    Detect outliers using z-score method
    """
    outliers_mask = np.zeros(len(df), dtype=bool)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for column in numeric_cols:
        if df[column].nunique() > 2:  # Skip binary columns
            z_scores = np.abs(stats.zscore(df[column], nan_policy='omit'))
            outliers_mask |= z_scores > n_std
    return outliers_mask


def preprocess_train(dataframe, features_to_keep, k=5, handle_outliers=True):
    """
    Enhanced preprocessing function with advanced feature handling
    """
    print("Starting preprocessing...")
    dataframe_selected = dataframe[features_to_keep].copy()
    
    # Drop columns with too many NaN values
    columns_to_drop = []
    for column in features_to_keep:
        nan_ratio = dataframe[column].isna().sum() / len(dataframe[column])
        if nan_ratio > 0.7:
            columns_to_drop.append(column)
    
    dataframe_selected.drop(columns=columns_to_drop, inplace=True)
    dataframe_selected = dataframe_selected.dropna(subset=["sii"])
    
    # Store IDs and target
    ids = dataframe_selected['id']
    sii = dataframe_selected['sii']
    
    # Prepare data for imputation
    train_data = dataframe_selected.drop(['id', 'sii'], axis=1)
    
    # Scale numeric features before imputation
    scaler = StandardScaler()
    numeric_cols = train_data.select_dtypes(include=[np.number]).columns
    train_data[numeric_cols] = scaler.fit_transform(train_data[numeric_cols])
    
    # Enhanced KNN imputation
    print(f"Performing KNN imputation with k={k}...")
    model_imputer = KNNImputer(n_neighbors=k, weights='distance')
    imputed_data = model_imputer.fit_transform(train_data)
    
    # Convert back to DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=train_data.columns)
    
    # Handle outliers if requested
    if handle_outliers:
        print("Detecting and handling outliers...")
        outliers_mask = detect_outliers(imputed_df)
        print(f"Found {sum(outliers_mask)} outliers")
        
        # Instead of removing outliers, clip them to acceptable ranges
        for col in numeric_cols:
            lower_bound = imputed_df[col].mean() - 3 * imputed_df[col].std()
            upper_bound = imputed_df[col].mean() + 3 * imputed_df[col].std()
            imputed_df[col] = imputed_df[col].clip(lower_bound, upper_bound)
    
    # Add back IDs and target
    imputed_df['id'] = ids.values
    imputed_df['sii'] = sii.values
    
    return imputed_df, columns_to_drop, model_imputer, scaler

In [25]:

def pipeline_train(dataframe_train, features_to_keep, k=3, optimize_hyperparams=True):
    """
    Enhanced training pipeline with advanced preprocessing and model training
    """
    # Preprocessing
    imputed_df, columns_dropped, knn, scaler = preprocess_train(
        dataframe_train, features_to_keep, k)
    
    # Prepare features and target
    X = imputed_df.drop(['id', 'sii'], axis=1)
    y = imputed_df['sii']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Train model
    best_model, avg_accuracy, best_params = train_xgboost(
        X_train, X_test, y_train, y_test, optimize_hyperparams)
    
    # Get feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return {
        'accuracy': avg_accuracy,
        'model': best_model,
        'columns_dropped': columns_dropped,
        'knn_imputer': knn,
        'scaler': scaler,
        'feature_importance': feature_importance,
        'best_params': best_params
    }

def find_best_k(train_data, features_to_keep, k_range=range(5, 33)):
    """
    Find the optimal k value for KNN imputation
    """
    results = []
    for k in k_range:
        print(f"\nTesting k={k}")
        pipeline_results = pipeline_train(
            train_data, features_to_keep, k, optimize_hyperparams=False)
        accuracy = pipeline_results['accuracy']
        results.append({'k': k, 'accuracy': accuracy})
        print(f'k={k}, Accuracy={accuracy * 100:.2f}%')
    
    best_result = max(results, key=lambda x: x['accuracy'])
    print(f"\nBest k for imputation: {best_result['k']} "
          f"with accuracy: {best_result['accuracy'] * 100:.2f}%")
    
    return best_result['k']


def train_xgboost(X_train, X_test, y_train, y_test, optimize_hyperparams=True):
    """
    Enhanced XGBoost training with optional hyperparameter optimization
    """
    if optimize_hyperparams:
        print("Optimizing hyperparameters...")
        study = create_study(direction='maximize')
        study.optimize(
            lambda trial: objective(trial, X_train, y_train, num_class=4),
            n_trials=50
        )
        best_params = study.best_params
        best_params.update({
            'objective': 'multi:softmax',
            'num_class': 4,
            'random_state': 42
        })
        print("Best parameters:", best_params)
    else:
        best_params = {
            'objective': 'multi:softmax',
            'num_class': 4,
            'learning_rate': 0.1,
            'max_depth': 6,
            'n_estimators': 100,
            'random_state': 42,
            'alpha': 0.0
        }
    
    # Use StratifiedKFold for better handling of imbalanced classes
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy_scores = []
    models = []
    
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold = X_train.iloc[train_index]
        X_val_fold = X_train.iloc[val_index]
        y_train_fold = y_train.iloc[train_index]
        y_val_fold = y_train.iloc[val_index]
        
        model = xgb.XGBClassifier(**best_params)
        
        # Train the model with eval_set but without early_stopping_rounds
        model.fit(
            X_train_fold, 
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            verbose=False
        )
        
        y_pred = model.predict(X_val_fold)
        accuracy = accuracy_score(y_val_fold, y_pred)
        accuracy_scores.append(accuracy)
        models.append(model)
    
    best_index = np.argmax(accuracy_scores)
    best_model = models[best_index]
    avg_accuracy = np.mean(accuracy_scores)
    
    return best_model, avg_accuracy, best_params

def objective(trial: Trial, X_train, y_train, num_class):
    """
    Optuna objective function for XGBoost hyperparameter optimization
    """
    param = {
        'objective': 'multi:softmax',
        'num_class': num_class,
        'tree_method': 'hist',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
    }
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, valid_idx in cv.split(X_train, y_train):
        X_train_fold = X_train.iloc[train_idx]
        y_train_fold = y_train.iloc[train_idx]
        X_valid_fold = X_train.iloc[valid_idx]
        y_valid_fold = y_train.iloc[valid_idx]
        
        model = xgb.XGBClassifier(**param, random_state=42)
        model.fit(
            X_train_fold, 
            y_train_fold,
            eval_set=[(X_valid_fold, y_valid_fold)],
            verbose=False
        )
        
        pred = model.predict(X_valid_fold)
        fold_score = accuracy_score(y_valid_fold, pred)
        scores.append(fold_score)
    
    return np.mean(scores)


In [26]:
def preprocess_test(test_data, features_to_keep, columns_to_drop, knn_model, scaler):
    """
    Preprocess test data consistently with training data
    
    Parameters:
    -----------
    test_data : pandas DataFrame
        The test dataset
    features_to_keep : list
        List of features to keep in the preprocessing
    columns_to_drop : list
        Columns that were dropped during training
    knn_model : KNNImputer
        Fitted KNN imputer from training
    scaler : StandardScaler
        Fitted scaler from training
    """
    print("Preprocessing test data...")
    
    # Select and drop columns consistently with training
    dataframe_selected = test_data[features_to_keep].copy()
    if columns_to_drop:
        dataframe_selected.drop(columns=columns_to_drop, inplace=True)
    
    # Store IDs
    ids = dataframe_selected['id']
    
    # Prepare data for imputation
    test_features = dataframe_selected.drop(['id'], axis=1)
    
    # Scale numeric features consistently with training
    numeric_cols = test_features.select_dtypes(include=[np.number]).columns
    test_features[numeric_cols] = scaler.transform(test_features[numeric_cols])
    
    # Apply KNN imputation
    print("Applying KNN imputation...")
    imputed_data = knn_model.transform(test_features)
    
    # Convert back to DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=test_features.columns)
    imputed_df['id'] = ids.values
    
    return imputed_df

def predict_test_data(test_data, model, pipeline_results, features_to_keep_test):
    """
    Complete pipeline for processing test data and making predictions
    
    Parameters:
    -----------
    test_data : pandas DataFrame
        The test dataset
    model : XGBClassifier
        Trained model
    pipeline_results : dict
        Results from training pipeline including preprocessors
    features_to_keep_test : list
        Features to use for test data
    
    Returns:
    --------
    tuple : (predictions, processed_test_data)
        Predictions and the processed test data
    """
    try:
        # Extract preprocessors from pipeline results
        knn_imputer = pipeline_results['knn_imputer']
        scaler = pipeline_results['scaler']
        columns_dropped = pipeline_results['columns_dropped']
        
        # Preprocess test data
        processed_test = preprocess_test(
            test_data,
            features_to_keep_test,
            columns_dropped,
            knn_imputer,
            scaler
        )
        
        # Prepare features for prediction
        test_features = processed_test.drop('id', axis=1)
        
        # Ensure all columns are numeric
        numeric_test = test_features.select_dtypes(include=[np.number])
        
        # Check for any remaining non-numeric columns
        non_numeric_cols = set(test_features.columns) - set(numeric_test.columns)
        if non_numeric_cols:
            print(f"Warning: Found non-numeric columns: {non_numeric_cols}")
            print("Converting to numeric...")
            test_features = test_features.apply(pd.to_numeric, errors='coerce')
        
        # Make predictions
        print("Making predictions...")
        predictions = model.predict(test_features)
        
        # Create results DataFrame
        results_df = pd.DataFrame({
            'id': processed_test['id'],
            'sii': predictions
        })
        
        print("Prediction complete!")
        return results_df, processed_test
        
    except Exception as e:
        print(f"Error in prediction pipeline: {str(e)}")
        raise


In [27]:
def run_complete_pipeline(train_data, test_data, features_to_keep_train, features_to_keep_test, k=19):
    """
    Run complete pipeline from training through prediction
    """
    # Train the model
    print("Training model...")
    pipeline_results = pipeline_train(train_data, features_to_keep_train, k=k, optimize_hyperparams=True)
    
    # Make predictions
    print("Processing test data and making predictions...")
    predictions_df, processed_test = predict_test_data(
        test_data,
        pipeline_results['model'],
        pipeline_results,
        features_to_keep_test
    )
    
    return predictions_df, pipeline_results, processed_test

In [ ]:
# Run the complete pipeline
predictions_df, pipeline_results, processed_test = run_complete_pipeline(
    train_data=train,
    test_data=test,
    features_to_keep_train=features_to_keep_train,
    features_to_keep_test=features_to_keep_test,
    k=20 
)

# Access results
print("\nModel Performance:")
print(f"Training Accuracy: {pipeline_results['accuracy']*100:.2f}%")
print("\nPrediction Summary:")
print(predictions_df['sii'].value_counts())

Training model...
Starting preprocessing...
Performing KNN imputation with k=20...


[I 2024-12-08 17:50:23,264] A new study created in memory with name: no-name-0b848c85-b70f-4000-a541-e1fd4603cac7


Detecting and handling outliers...
Found 210 outliers
Optimizing hyperparameters...


[I 2024-12-08 17:50:25,125] Trial 0 finished with value: 0.5950607608956877 and parameters: {'learning_rate': 0.2964191484933706, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.890469399504168, 'colsample_bytree': 0.740481618479278, 'gamma': 1.8203736329394578, 'alpha': 0.2037532099688386, 'n_estimators': 153}. Best is trial 0 with value: 0.5950607608956877.
[I 2024-12-08 17:50:28,982] Trial 1 finished with value: 0.5850119640972593 and parameters: {'learning_rate': 0.11841886701706443, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.9495389064524977, 'colsample_bytree': 0.7902224229634198, 'gamma': 0.4543195800451971, 'alpha': 0.01002029151676504, 'n_estimators': 239}. Best is trial 0 with value: 0.5950607608956877.
[I 2024-12-08 17:50:31,172] Trial 2 finished with value: 0.6037532783716288 and parameters: {'learning_rate': 0.07155958652369467, 'max_depth': 12, 'min_child_weight': 7, 'subsample': 0.8994887001969856, 'colsample_bytree': 0.9677639825641907, 'gamma': 2.80080